# ELASTİCNET REGRESSİON

In [6]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet  #elasticNet model seçimi
from sklearn.linear_model import RidgeCV, ElasticNetCV   #ElasticNET cross validation seçini
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [16]:
#Veri seti
df = pd.read_csv("beyzbol_veri.csv")

df = df.dropna()   #Şuanda veri önişleme konu ile alakalı olmadığından işleme alınmamış eksik değerler uçurulmuştur.

dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])  #One-hat dönüşümü degisken_standart_donusumu.jpynb

y = df["Salary"]  #oyuncuların değerleri ile tahmin yapılacak. Bağımlı değişken

X_ = df.drop(["Salary","League", "Division", "NewLeague"], axis = 1).astype("float64") 
# df içinden league division ve newleague değişkenleri uçurarak one-hat dönüşümü yapılan bu değişkenler kullanılacak.

X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis = 1) #bağımsız değişkenler.

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.25, random_state = 42)

In [17]:
enet_model = ElasticNet().fit(X_train, y_train)  #model test verileri üzerinden oluşturuldu.

C:\Users\Mutlu\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1512791.8875564984, tolerance: 866.4061181073539
  positive)


In [18]:
enet_model.coef_

array([-2.03049935e+00,  9.76161878e+00, -1.65614092e+01, -5.54317500e+00,
        5.22419006e+00,  4.14452934e+00, -1.13319398e+01, -1.16175385e-01,
       -3.99883908e-01,  2.94397685e+00,  2.26755267e+00,  3.28021123e-01,
       -1.08412588e+00,  2.83701826e-02,  9.02700772e-02,  1.79822396e+00,
        4.81237518e+01, -3.10856340e+01,  4.13473782e+01])

In [19]:
enet_model.intercept_

92.11631586588328

In [20]:
y_test_pred = enet_model.predict(X_test)

In [22]:
np.sqrt(mean_squared_error(y_test, y_test_pred))  #RMSE hatası

373.6203977175176

In [26]:
#r2 score açıklanabilirlik
r2_score(y_test, y_test_pred)

0.3760059144280571

### Model Tuning

In [27]:
lamdalar = 10*np.linspace(10, -2, 10)*0.5

In [36]:
enet_model = ElasticNetCV(alphas = lamdalar,  cv = 10).fit(X_train, y_train)
#cross validation 10 katlı. lamdalar listesinnin içerisinden en uygun alpha değeri seçilmesi

C:\Users\Mutlu\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 898952.3022686343, tolerance: 525.9612201701586
  tol, rng, random, positive)
C:\Users\Mutlu\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 703382.3018641747, tolerance: 525.9612201701586
  tol, rng, random, positive)
C:\Users\Mutlu\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 56884.9676967815, tolerance: 525.9612201701586
  tol, rng, random, positive)
C:\Users\Mutlu\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might

In [33]:
enet_model.alpha_

50.0

In [34]:
enet_model.coef_

array([-0.98778859,  5.8382303 , -3.05187387, -2.44599695,  0.38652773,
        2.66783035,  0.        , -0.30527029, -0.0201198 ,  0.11792298,
        2.02540048,  1.44966429, -0.80676275,  0.04958727,  0.27523164,
        0.09987512,  0.46794675, -0.22838272,  0.25219594])

In [39]:
y_pred = enet_model.predict(X_test)

In [40]:
np.sqrt(mean_squared_error(y_test, y_pred))

357.9060985940664

In [46]:
?ElasticNet
#l1_ratio ifadesinin anlamı : eğer değer 0 olursa L2 cezalandırılması, 1 olursa L1 cezalandırılması yapıldığı anlamına gelir.
# L1 ve L2 nin göreceli olarak etkilerini ifade etmektedir. İleri seviyede bu etki değiştirilir.

Init signature:
ElasticNet(
    alpha=1.0,
    l1_ratio=0.5,
    fit_intercept=True,
    normalize=False,
    precompute=False,
    max_iter=1000,
    copy_X=True,
    tol=0.0001,
    warm_start=False,
    positive=False,
    random_state=None,
    selection='cyclic',
)
Docstring:     
Linear regression with combined L1 and L2 priors as regularizer.

Minimizes the objective function::

        1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

If you are interested in controlling the L1 and L2 penalty
separately, keep in mind that this is equivalent to::

        a * L1 + b * L2

where::

        alpha = a + b and l1_ratio = a / (a + b)

The parameter l1_ratio corresponds to alpha in the glmnet R package while
alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio
= 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable,
unless you supply your own sequence of alpha.

Read more i